In [1]:
import minnetonka as mn

## Building a model in Minnetonka

A simple banking model, with savings, interest, and an interest rate.

In [2]:
with mn.model() as m:
    mn.constant('Rate', 0.02)
    mn.stock('Savings', lambda interest: interest, ('Interest',), 1000)
    mn.variable('Interest', lambda savings, rate: savings * rate, 'Savings', 'Rate')

Let's look at the initial amounts of **Savings** and **Interest**.

Variable values are accessed by indexing with an empty string, for reasons that will be made clear later.

In [3]:
m['Savings']['']

1000

In [4]:
m['Interest']['']

20.0

Advance the model one year.

In [5]:
m.step()

In [6]:
m['Savings']['']

1020.0

Advance it another year.

In [7]:
m.step()
m['Savings']['']

1040.4

Advance ten years.

In [8]:
m.step(10)
m['Savings']['']

1268.2417945625452

Reset back to the beginning

In [9]:
m.reset()
m['Savings']['']

1000

In [10]:
m.step()
m['Savings']['']

1020.0

## Treatments and TIME

Same model, but now with two treatments instead of the default (empty string) treatment. 
One treatment will be low interest, the other high interest. Also we will use a Python 
variables, so we can write **Savings** instead of **m['Savings']**.

In [11]:
with mn.model(treatments=['Low interest', 'High interest']) as m:
    mn.constant('Rate', mn.PerTreatment({'Low interest': 0.01, 'High interest': 0.03}))
    Savings = mn.stock('Savings', lambda interest: interest, ('Interest',), 1000)
    mn.variable('Interest', lambda savings, rate: savings * rate, 'Savings', 'Rate')

Initially **Savings** has the same amount in both treatments.

Variable amounts are accessed by indexing with the treatment name. 

In [12]:
Savings['Low interest']

1000

In [13]:
Savings['High interest']

1000

After a single year, **Savings** diverges.

In [14]:
m.step()
Savings['Low interest']

1010.0

In [15]:
Savings['High interest']

1030.0

More divergence after a few more years.

In [16]:
m.step(4)
Savings['Low interest']

1051.0100501

In [17]:
Savings['High interest']

1159.2740743

Five years have passed.

In [18]:
m.TIME

5

After reset, TIME is 0

In [19]:
m.reset()
m.TIME

0

## Different timestep

Same model, but now with a time step that is less than one unit. The time unit remains a year, but the time step is a quarter. 

Also **Rate** and **Interest** are now Python variables, for easy access. 

In [20]:
with mn.model(treatments=['Low interest', 'High interest'], timestep=0.25) as m:
    Rate = mn.constant('Rate', mn.PerTreatment({'Low interest': 0.01, 'High interest': 0.03}))
    Savings = mn.stock('Savings', lambda interest: interest, ('Interest',), 1000)
    Interest = mn.variable('Interest', lambda savings, rate: savings * rate, 'Savings', 'Rate')

Same at the start

In [21]:
Savings['Low interest']

1000

In [22]:
Savings['High interest']

1000

But step() now advances only a quarter.

In [23]:
m.step()
m.TIME

0.25

In [24]:
Savings['Low interest']

1002.5

In [25]:
Savings['High interest']

1007.5

Note that **Interest** is still defined on an annual basis, even though only 25% of the interest is applied each quarter.

In [26]:
Interest['Low interest']

10.025

In [27]:
Interest['High interest']

30.224999999999998

Advance 20 time steps (5 years).

In [28]:
m.step(20)
m.TIME

5.25

In [29]:
Savings['Low interest']

1053.8335170395887

In [30]:
Savings['High interest']

1169.8930233704716

## Same model, but a change outside the model logic.

Reset and advance two years.

In [31]:
m.reset()
m.step(8)
m.TIME

2.0

The interest rate for the high interest treatment changes. This change is outside the logic of the model. Note that the change is only to the amount in a single treatment. The low interest treatment is unaffected.

In practice changes outside the logic of the model could be due to user intervention (as in this case), or could be due to real world integration, integration with other models, or integration with other software.

In [32]:
Rate['High interest'] = 0.02

Propagate the change through the rest of the model. As [explained in the documentation](https://bridgeland.github.io/minnetonka/index.html?highlight=recalculate#minnetonka.minnetonka.Model.recalculate), recalculation is only necessary when the amount of a variable (or constant or stock) is changed explicitly, outside of the model logic. The variables that depend on that changed variable will take amounts that do not reflect the changes, at least until the model is stepped. If that is not appropriate, a call to recalculate() will calculate new updated amounts for all those dependent variables.

In [33]:
m.recalculate()

**Interest** reflects the changed rate.

In [34]:
Interest['High interest']

21.231976956365504

Advance 13 time steps (3.25 years), to the same time as before the reset.

In [35]:
m.step(13)
m.TIME

5.25

After 5.25 years, **Savings** reflects the lowered rate.

In [36]:
Savings['High interest']

1132.711321538728

The low interest treatment is unaffected by the change.

In [37]:
Savings['Low interest']

1053.8335170395887

On reset, the high interest rate reverts to its model value.

In [39]:
m.reset()
Rate['High interest']

0.03

## Numpy arrays and other amounts

Instead of integers and floats, Minnetonka variables can take amounts that are any Python object, anything except a string. Let's try the same model with numpy arrays.

In [40]:
import numpy as np

In [41]:
initial_savings = np.array([1000, 2000, 3000, 4000])

**Savings** takes an initial amount that is a numpy array. Note that the rest of the model is unchanged.

In [42]:
with mn.model(treatments=['Low interest', 'High interest'], timestep=0.25) as m:
    Rate = mn.constant('Rate', mn.PerTreatment({'Low interest': 0.01, 'High interest': 0.03}))
    Savings = mn.stock('Savings', lambda interest: interest, ('Interest',), initial_savings)
    Interest = mn.variable('Interest', lambda savings, rate: savings * rate, 'Savings', 'Rate')

The amount of **Interest** is also a numpy array in this model.

In [43]:
Interest['High interest']

array([ 30.,  60.,  90., 120.])

Advance 5 years.

In [44]:
m.step(20)

In [45]:
Savings['Low interest']

array([1051.20550328, 2102.41100656, 3153.61650984, 4204.82201313])

In [46]:
Savings['High interest']

array([1161.1841423 , 2322.36828461, 3483.55242691, 4644.73656921])